# Pattern recognition CW2

In [6]:
from scipy.io import loadmat
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
import time
from sklearn.cluster import KMeans
import metric_learn

### Import file with 6 main components:

In [2]:
""" 
camId : which camera was used to get the shot (1 or 2)
filelist: names of the images (with format x_label_camId_index.png)
labels: class of the image (which person's image is it?)
query_idx: indexes of test set
gallery_idx: indexes of test set used for kNN 
train_idx: indexes of training and validation set
"""   
train_idxs = loadmat('cuhk03_new_protocol_config_labeled.mat')

### Import file with the feature vectors of all images:

In [3]:
import json
with open('PR_data/feature_data.json', 'r') as f:
    features = json.load(f)
features_np = np.array(features) #list of features converted to an array 
features_np.shape

(14096, 2048)

In [4]:
gallery_indices = train_idxs['gallery_idx'].flatten() - 1
gallery_images = features_np[gallery_indices]
gallery_labels = train_idxs['labels'][gallery_indices]

query_indices = train_idxs['query_idx'].flatten() - 1
query_images = features_np[query_indices]
query_labels = train_idxs['labels'][query_indices]

training_indices = train_idxs['train_idx'].flatten() - 1
training_images = features_np[training_indices]
training_labels = train_idxs['labels'][training_indices]

## kNN Classification for query set on gallery set

In [ ]:
def calculate_recall_index(value, recall): #only used in calcAP function below
    for index, recall_val in enumerate(recall):
        if recall_val >= value:
            return index
    
def calcAP(true_label, rank_k_labels):
    correct_list = [true_label == this_label for this_label in rank_k_labels]    
    precision = []
    recall = []
    for i in range(len(rank_k_labels)):
        precision.append(sum(correct_list[:i+1])/len(correct_list[:i+1]))
        recall.append(sum(correct_list[:i+1])/sum(correct_list))
    max_list = []
    for i in range(11):
        recall_index = calculate_recall_index(i/10, recall)
        max_list.append(max(precision[recall_index:]))
    print(max_list)
    return np.mean(max_list)

In [25]:
def remove_indices(n_indices, query_index):
    query_label_and_camId = (train_idxs['labels'][query_index][0], train_idxs['camId'][query_index][0])
    list_of_gallery_label_and_camId = [(label[0], camId[0]) for label, camId in zip(train_idxs['labels'][n_indices], train_idxs['camId'][n_indices])] 
    final_list = list(filter(lambda a: a != query_label_and_camId, list_of_gallery_label_and_camId))
    return [tup[0] for tup in final_list]

def get_accuracy(k, N):
    list_of_truths = []
    AP_vals = []
    start = time.time() #time tracking - start time of process
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1)
    neigh.fit(gallery_images)   #new_gallery_labels)
    for index, query_index in enumerate(query_indices.tolist()):
        if (index % 100 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, 2048))
        forLoopStart = time.time()
        topN_gallery_indices = (gallery_indices[N_indices[0]]).tolist()
        reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
        AP = calcAP(query_labels[index][0], reduced_topN_labels[:k])
        AP_vals.append(AP)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    #print(acc)
    return acc, np.mean(AP_vals)

In [314]:
N=100
k = [1, 5, 10]
results = []
for k_val in k: 
    print("For rank ", k_val, ": ")
    accuracy, mAP = get_accuracy(k_val, N)
    tup = (k_val, accuracy, mAP)
    results.append(tup)
    print(tup)

For rank  1 : 
Index:  0  & time taken:  0.6711568832397461
Index:  100  & time taken:  11.172960996627808
Index:  200  & time taken:  21.65892195701599
Index:  300  & time taken:  32.16593790054321
Index:  400  & time taken:  42.716115951538086
Index:  500  & time taken:  53.33190989494324
Index:  600  & time taken:  63.814366817474365
Index:  700  & time taken:  74.30272483825684
Index:  800  & time taken:  84.79265689849854
Index:  900  & time taken:  95.2760329246521
Index:  1000  & time taken:  105.76963686943054
Index:  1100  & time taken:  116.35101675987244
Index:  1200  & time taken:  126.84344291687012
Index:  1300  & time taken:  137.45032286643982
0.47
For rank  5 : 
Index:  0  & time taken:  0.6394927501678467
Index:  100  & time taken:  11.25248098373413
Index:  200  & time taken:  21.72057294845581
Index:  300  & time taken:  32.267863035202026
Index:  400  & time taken:  42.73586893081665
Index:  500  & time taken:  53.28752779960632
Index:  600  & time taken:  63.81535

### Results for kNN: 

k = 1, Accuracy = 0.47
k = 5, Accuracy = 0.6685714285714286
k = 10, Accuracy = 0.7492857142857143

## K-means

In [5]:
def img_indices_in_cluster(c, index_and_cluster): 
    gallery_indices_in_cluster = [pair[0] for pair in index_and_cluster if pair[1] == c]
    return gallery_indices_in_cluster

def img_cluster(k, N, c_g_indices, query_index, index):
    list_of_truths = []
    neigh = NearestNeighbors(n_neighbors=N, n_jobs=-1, metric='euclidean')
    neigh.fit(features_np[c_g_indices])   #new_gallery_labels)
    N_distances, N_indices = neigh.kneighbors(query_images[index].reshape(1, -1))
    topN_c_g_indices = (gallery_indices[N_indices[0]]).tolist()
    reduced_topN_labels = remove_indices(topN_c_g_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

def knn_clustering(k, N, index_and_cluster):
    class_success = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(query_indices.tolist()[:700]):
        if (index % 200 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        cluster = kmeans.predict(query_images[index].reshape(1,-1))
        cluster_gallery_indices = img_indices_in_cluster(cluster, index_and_cluster)
        class_success.append(img_cluster(k, N, cluster_gallery_indices, query_index, index))
    
    #over-all accuracy
    acc = sum(class_success)/len(class_success)
    print(acc)
    return acc

In [47]:
total_number_of_labels = len(np.unique(gallery_labels.flatten()))

acc_kmeans = []
clusters = [1, 3, 10] 
clusters2 = [100, total_number_of_labels]
for n_clusters in clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(gallery_images)
    index_and_cluster = [(index, label) for index, label in zip(gallery_indices, kmeans.labels_)]
    k = [1, 5, 10]
    N = 100
    for k_val in k:
        acc_kmeans.append(knn_clustering(k_val, N, index_and_cluster)) 

Index:  0  & time taken:  4.506111145019531e-05
Index:  200  & time taken:  150.7624750137329
Index:  400  & time taken:  301.3440308570862
Index:  600  & time taken:  461.696977853775
0.49714285714285716
Index:  0  & time taken:  4.696846008300781e-05
Index:  200  & time taken:  156.14981603622437
Index:  400  & time taken:  307.5661029815674
Index:  600  & time taken:  461.1602611541748
0.69
Index:  0  & time taken:  5.0067901611328125e-05
Index:  200  & time taken:  153.74979496002197
Index:  400  & time taken:  308.7922320365906
Index:  600  & time taken:  465.1946897506714
0.7742857142857142
Index:  0  & time taken:  4.792213439941406e-05
Index:  200  & time taken:  51.17026996612549
Index:  400  & time taken:  104.5521879196167
Index:  600  & time taken:  155.86516094207764
0.0
Index:  0  & time taken:  7.796287536621094e-05
Index:  200  & time taken:  51.24071407318115
Index:  400  & time taken:  104.89248394966125
Index:  600  & time taken:  156.0232548713684
0.0014285714285714

In [ ]:
acc_kmeans

In [ ]:
acc_kmeans2 = []
clusters = [100, total_number_of_labels]
for n_clusters in clusters:
    kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    kmeans.fit(gallery_images)
    index_and_cluster = [(index, label) for index, label in zip(gallery_indices, kmeans.labels_)]
    k = [1, 5, 10]
    N = 50
    for k_val in k:
        acc_kmeans2.append(knn_clustering(k_val, N, index_and_cluster)) 

### Mahalanobi Distance

In [26]:
S = np.cov(np.transpose(training_images))
A = np.linalg.inv(S)

In [14]:
# setting up LMNN
lmnn = metric_learn.LMNN(k=1)
# fit the data!
lmnn.fit(training_images, training_labels)
# transform our input space
A = lmnn.metric()

In [15]:
S = np.linalg.inv(A)

In [31]:
list_of_truths = []
i = 0
N = 50
k = 1

start = time.time() #time tracking - start time of process
for q_index, q_image in zip(query_indices, query_images): 
    print("Index: ", i, " & time taken: ", time.time() - start)
    query_distances = []
    for g_index, g_image in zip(gallery_indices, gallery_images): 
        image_diff = q_image - g_image
        query_distances.append(tuple([g_index, np.linalg.multi_dot([np.transpose(image_diff), S, image_diff])]))
        #query_distances.append(tuple([g_index, np.dot(np.dot(np.transpose(image_diff), S), image_diff)]))
    query_distances_sorted = sorted(query_distances,key=lambda x:(x[1]))
    N_gallery_indices = [tup[0] for tup in query_distances_sorted][:N]
    reduced_N_gallery_labels = remove_indices(N_gallery_indices, q_index)
    if query_labels[i][0] in reduced_N_gallery_labels[:k]:
        list_of_truths.append(True)
    else:
        list_of_truths.append(False)
        
    if i == 20: 
        break
    i +=1
acc = sum(list_of_truths)/len(list_of_truths) 

Index:  0  & time taken:  0.0003781318664550781
Index:  1  & time taken:  12.34682297706604
Index:  2  & time taken:  27.84221315383911
Index:  3  & time taken:  43.7835910320282
Index:  4  & time taken:  57.16001796722412
Index:  5  & time taken:  71.11046099662781
Index:  6  & time taken:  85.21257090568542
Index:  7  & time taken:  97.4102931022644
Index:  8  & time taken:  109.49920701980591
Index:  9  & time taken:  121.57118320465088
Index:  10  & time taken:  133.7288088798523
Index:  11  & time taken:  145.9414541721344
Index:  12  & time taken:  158.0778648853302
Index:  13  & time taken:  170.14783310890198
Index:  14  & time taken:  182.60380911827087
Index:  15  & time taken:  194.58802318572998
Index:  16  & time taken:  206.63306999206543
Index:  17  & time taken:  218.6304099559784
Index:  18  & time taken:  230.8930628299713
Index:  19  & time taken:  243.50001406669617
Index:  20  & time taken:  255.6379430294037


In [28]:
gallery_images.shape

(5328, 2048)

In [69]:
mean_gallery_images = gallery_images.mean(axis=0)     # to take the mean of each column (colummn -> feature)
mean_gallery_images.shape

(2048,)

In [79]:
query_set_distances = [np.sqrt(np.dot(np.dot(np.transpose(query_image-mean_gallery_images),A), query_image-mean_gallery_images))
                      for query_image in query_images]
np.array(query_set_distances).shape

(1400,)

Getting distances using kNN and Mahalanobi Distance

In [31]:
N = 100
neigh_md = NearestNeighbors(n_neighbors=N,metric='mahalanobis', 
            metric_params= {'V': np.cov(np.transpose(training_images))})
neigh_md.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='mahalanobis',
         metric_params={'V': array([[ 0.12217,  0.00295, ..., -0.00943, -0.01215],
       [ 0.00295,  0.23106, ...,  0.00179, -0.01196],
       ...,
       [-0.00943,  0.00179, ...,  0.32026,  0.09041],
       [-0.01215, -0.01196, ...,  0.09041,  0.15056]])},
         n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [117]:
def get_accuracy_mahalanobi(k, N):
    list_of_truths = []
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        N_distances, N_indices = neigh_md.kneighbors(query_images[index].reshape(1, 2048), N)
        topN_gallery_images = gallery_images[N_indices[0]]
        forLoopStart = time.time()
        topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
        reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
        if query_labels[index][0] in reduced_topN_labels[:k]:
            list_of_truths.append(True)
        else:
            list_of_truths.append(False)
    #over-all accuracy  
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    return acc

In [118]:
N = 100
k = [1, 10, 100]
acc_k2 = []
for k_val in k: 
    acc = get_accuracy_mahalanobi(k_val, N)
    acc_k2.append(acc)

Index:  0  & time taken:  8.082389831542969e-05
Index:  50  & time taken:  1194.6821658611298
Index:  100  & time taken:  2801.5326719284058
Index:  150  & time taken:  4013.9011237621307
Index:  200  & time taken:  5203.121114730835
Index:  250  & time taken:  6391.557485818863
Avg Accuracy is:  0.13666666666666666
Index:  0  & time taken:  7.224082946777344e-05
Index:  50  & time taken:  1187.0171492099762
Index:  100  & time taken:  2373.66286110878
Index:  150  & time taken:  3560.3639900684357
Index:  200  & time taken:  4746.637277126312
Index:  250  & time taken:  5932.8617470264435
Avg Accuracy is:  0.2833333333333333
Index:  0  & time taken:  9.083747863769531e-05
Index:  50  & time taken:  1186.3573276996613
Index:  100  & time taken:  2372.715752840042
Index:  150  & time taken:  3559.498749732971
Index:  200  & time taken:  4745.957722663879
Index:  250  & time taken:  5932.393654823303
Avg Accuracy is:  0.4533333333333333


In [112]:
acc_k

[0.09090909090909091, 0.18181818181818182, 0.36363636363636365]

In [119]:
acc_k2

[0.13666666666666666, 0.2833333333333333, 0.4533333333333333]

### Cosine Distance

In [120]:
N = 100
neigh_c = NearestNeighbors(n_neighbors=N,metric='cosine')
neigh_c.fit(gallery_images) 

NearestNeighbors(algorithm='auto', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=1, n_neighbors=100, p=2, radius=1.0)

In [126]:
def get_acc_gen(k, N, index, query_index):
    N_distances, N_indices = neigh_c.kneighbors(query_images[index].reshape(1, 2048), N)
    topN_gallery_images = gallery_images[N_indices[0]]
    forLoopStart = time.time()
    topN_gallery_indices = train_idxs['gallery_idx'][N_indices[0]].flatten().tolist()
    reduced_topN_indices, reduced_topN_labels = remove_indices(topN_gallery_indices, query_index)
    return (query_labels[index][0] in reduced_topN_labels[:k])

k = [1, 10, 100]
acc_k3 = []
for k_val in k:
    list_of_truths = [] 
    start = time.time() #time tracking - start time of process
    for index, query_index in enumerate(train_idxs['query_idx'].flatten().tolist()[:300]):
        if (index % 50 == 0):
            print("Index: ", index, " & time taken: ", time.time() - start)
        truth_false = get_acc_gen(k_val, N, index, query_index)
        list_of_truths.append(truth_false)
    acc = sum(list_of_truths)/len(list_of_truths)
    print("Avg Accuracy is: ", acc)
    acc_k3.append(acc)

Index:  0  & time taken:  7.700920104980469e-05
Index:  50  & time taken:  4.227119207382202
Index:  100  & time taken:  8.423174142837524
Index:  150  & time taken:  12.630351066589355
Index:  200  & time taken:  16.83267116546631
Index:  250  & time taken:  21.04805612564087
Avg Accuracy is:  0.49
Index:  0  & time taken:  5.1975250244140625e-05
Index:  50  & time taken:  4.1925787925720215
Index:  100  & time taken:  8.413705825805664
Index:  150  & time taken:  12.6040678024292
Index:  200  & time taken:  16.792085886001587
Index:  250  & time taken:  20.98653793334961
Avg Accuracy is:  0.74
Index:  0  & time taken:  5.3882598876953125e-05
Index:  50  & time taken:  4.198216199874878
Index:  100  & time taken:  8.467583894729614
Index:  150  & time taken:  12.67863917350769
Index:  200  & time taken:  16.884459972381592
Index:  250  & time taken:  21.093831062316895
Avg Accuracy is:  0.91
